In [120]:
import warnings
warnings.simplefilter("ignore")

In [121]:
import pandas as pd
import numpy as np
import duckdb
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Load data

olist_customers_dataset

In [122]:
df_customer = pd.read_csv('./dataset/olist_customers_dataset.csv')

In [123]:
df_customer.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [124]:
df_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


olist_geolocation_dataset

In [125]:
df_geo = pd.read_csv('./dataset/olist_geolocation_dataset.csv')

In [126]:
df_geo.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [127]:
df_geo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  int64  
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geolocation_city             1000163 non-null  object 
 4   geolocation_state            1000163 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 38.2+ MB


olist_order_items_dataset

In [128]:
df_order_item = pd.read_csv('./dataset/olist_order_items_dataset.csv')

In [129]:
df_order_item.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [130]:
df_order_item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


In [131]:
df_order_item['shipping_limit_date'] = pd.to_datetime(df_order_item['shipping_limit_date'])

olist_order_payments_dataset

In [132]:
df_order_payment = pd.read_csv('./dataset/olist_order_payments_dataset.csv')

In [133]:
df_order_payment.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [134]:
df_order_payment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


olist_order_reviews_dataset

In [135]:
df_order_review = pd.read_csv('./dataset/olist_order_reviews_dataset.csv')

In [136]:
df_order_review.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [137]:
df_order_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [138]:
df_order_review['review_creation_date'] = pd.to_datetime(df_order_review['review_creation_date'])
df_order_review['review_answer_timestamp'] = pd.to_datetime(df_order_review['review_answer_timestamp'])

olist_orders_dataset

In [139]:
df_order = pd.read_csv('./dataset/olist_orders_dataset.csv')

In [140]:
df_order.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [141]:
df_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [142]:
df_order['order_purchase_timestamp'] = pd.to_datetime(df_order['order_purchase_timestamp'])
df_order['order_approved_at'] = pd.to_datetime(df_order['order_approved_at'])
df_order['order_delivered_carrier_date'] = pd.to_datetime(df_order['order_delivered_carrier_date'])
df_order['order_delivered_customer_date'] = pd.to_datetime(df_order['order_delivered_customer_date'])
df_order['order_estimated_delivery_date'] = pd.to_datetime(df_order['order_estimated_delivery_date'])

olist_products_dataset

In [143]:
df_product = pd.read_csv('./dataset/olist_products_dataset.csv')

In [144]:
df_product.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [145]:
df_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


olist_sellers_dataset

In [146]:
df_seller = pd.read_csv('./dataset/olist_sellers_dataset.csv')

In [147]:
df_seller.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [148]:
df_seller.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB


product_category_name_translation

In [149]:
df_cat_name = pd.read_csv('./dataset/product_category_name_translation.csv')

In [150]:
df_cat_name.head()

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [151]:
df_cat_name.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   product_category_name          71 non-null     object
 1   product_category_name_english  71 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


olist_age

In [152]:
df_age = pd.read_csv('./dataset/olist_age.csv')

In [153]:
df_age.head()

,customer_id,age
0,00012a2ce6f8dcda20d059ce98491703,61
1,000161a058600d5901f007fab4c27140,49
2,0001fd6190edaaf884bcaf3d49edf079,71
3,0002414f95344307404f0ace7a26f1d5,68
4,000379cdec625522490c315e70c7a9fb,61


In [154]:
df_age.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98666 entries, 0 to 98665
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_id  98666 non-null  object
 1   age          98666 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.5+ MB


# 2. Feature engineering

1. customer_id

In [155]:
df_customer_unique_id = duckdb.query("""
select distinct customer_unique_id
from df_customer
""").to_df()

In [156]:
df_customer_unique_id

,customer_unique_id
0,290c77bc529b7ac935b93aa66c333dc3
1,060e732b5b29e8181a18229c7b0b2b5e
2,5c2991dbd08bbf3cf410713c4de5a0b5
3,b6e99561fe6f34a55b0b7da92f8ed775
4,5f102dd37243f152aec3607970aad100
...,...
96091,065b9e6198f11dbdf7fc1250f9e08afc
96092,c64ede6d0ae8901b1b6fb03528c1b7e6
96093,5a1680640123fc80e801301a2fb57395
96094,e9f50caf99f032f0bf3c55141f019d99


2. number of sub account

In [157]:
df_customer_no_acc = duckdb.query("""
select customer_unique_id, count(customer_id) as no_account
from df_customer
group by customer_unique_id
order by no_account desc
""").to_df()

In [158]:
df_customer_no_acc

,customer_unique_id,no_account
0,8d50f5eadf50201ccdcedfb9e2ac8455,17
1,3e43e6105506432c953e165fb2acf44c,9
2,6469f99c1f9dfae7733b25662e7f1782,7
3,ca77025e7201e3b30c44b472ff346268,7
4,1b6c7548a2a1f9037c1fd3ddfed95f33,7
...,...,...
96091,a662a415dbbb5a154b3d38aa4fc86e57,1
96092,592134387e7f13de6a9e38eb7d412b3d,1
96093,3c8d75c7b0acfda04e92d3dca43ccee7,1
96094,fb354969e06f2093c0083cbfbb91864e,1


3. age

In [159]:
df_customer_age = duckdb.query("""
select b.customer_unique_id
, min(a.age) age
from df_age a
left join df_customer b
on a.customer_id = b.customer_id
group by b.customer_unique_id
""").to_df()

In [160]:
df_customer_age

,customer_unique_id,age
0,0b83f73b19c2019e182fd552c048a22c,61
1,e7897290aea0805ab175017e3f60c3e7,64
2,48efc7def11f9a644e1993a3d97361d2,69
3,490c854539b21598cfbbac518ca25788,73
4,392c8818b71b14227a7c701e7d54a3a1,50
...,...,...
95415,4b66b83ef70065c79f4ebe6875902f3f,56
95416,3de473dafcdf00614810c9eb37ee2cd7,31
95417,587b326ba3bf8aa4d3e50fb1f38ea79f,69
95418,ccbc9ad1e581dceecfb8638696ef5475,68


4. customer address

In [161]:
df_customer_address = duckdb.query("""
select customer_unique_id
, case when customer_state in ('AC', 'AP', 'AM', 'PA', 'RO', 'RR', 'TO') then 1 else 0 end north
, case when customer_state in ('AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE') then 1 else 0 end northeast
, case when customer_state in ('DF', 'GO', 'MT', 'MS') then 1 else 0 end central_west
, case when customer_state in ('ES', 'MG', 'RJ', 'SP') then 1 else 0 end southeast
, case when customer_state in ('PR', 'RS', 'SC') then 1 else 0 end south
from
    (
    select customer_unique_id, customer_state, customer_city, customer_zip_code_prefix
    , row_number() over(partition by customer_unique_id order by customer_zip_code_prefix) rn
    from df_customer
    ) a
where rn = 1
""").to_df()

In [162]:
df_customer_address

,customer_unique_id,north,northeast,central_west,southeast,south
0,000ed48ceeb6f4bf8ad021a10a3c7b43,0,0,0,1,0
1,001deb796b28a3a128d6113857569aa4,0,0,0,1,0
2,00256bf7b05c08f6a5849df3d0a64f1f,0,0,0,1,0
3,002eb8b96288f3804e21d2777ac29895,0,0,0,1,0
4,0033f4a2887e07e25aa1204d482f41ba,0,1,0,0,0
...,...,...,...,...,...,...
96091,fe0617cc03bb1fdefeb9e76e7d316548,0,0,0,1,0
96092,fe163b538b9f14ce8e554c1ebcc66527,0,0,0,1,0
96093,fe36ffe7c25e22b2276fd6e9f0056e24,0,0,0,1,0
96094,fe40c8ed5d902c8ec1032044a223b285,0,0,0,1,0


5. recency

In [163]:
df_customer_recency = duckdb.query("""
select b.customer_unique_id
, max(cast(order_purchase_timestamp as datetime)) lastest_txn
, min(datediff('day', cast(order_purchase_timestamp as datetime), cast('2018-11-01' as datetime))) as day_after_last_pur
from df_order a
left join df_customer b on a.customer_id = b.customer_id
where a.order_status not in ('unavailable','canceled')
and a.order_purchase_timestamp is not null
group by b.customer_unique_id
""").to_df()

In [164]:
df_customer_recency

,customer_unique_id,lastest_txn,day_after_last_pur
0,57b2a98a409812fe9618067b6b8ebe4f,2017-11-16 19:29:02,350
1,21f748a16f4e1688a9014eb3ee6fa325,2018-01-29 20:32:08,276
2,a96d5cfa0d3181817e2b946f921ea021,2017-10-15 11:08:48,382
3,e9dd12dca17352644a959d9dea133935,2018-04-18 09:42:53,197
4,e079b18794454de9d2be5c12b4392294,2017-06-14 18:52:29,505
...,...,...,...
94985,c8a9a5a7906ab84c7dcf742761a0616c,2017-09-13 19:58:52,414
94986,c7d1c1a9792b4c5b53f6e88807ec88f6,2017-11-27 13:53:44,339
94987,04c8f10e74b113ab86550eab24b89855,2018-07-18 14:37:24,106
94988,8ee2c84bb387d1e4fbac648a5d964581,2018-03-13 14:53:47,233


6. frequency

In [165]:
duckdb.query("""
select max(cast(order_purchase_timestamp as datetime)) max_date from df_order
""").to_df()

,max_date
0,2018-10-17 17:30:18


In [166]:
df_customer_frequency = duckdb.query("""
select b.customer_unique_id
, count(distinct order_id) no_order
, max(datediff('month',cast(order_purchase_timestamp as datetime),cast('2018-11-01' as datetime))) account_age_month
, count(distinct order_id)/
min(datediff('month',cast(order_purchase_timestamp as datetime),cast('2018-11-01' as datetime))) order_per_month
from df_order a
left join df_customer b
on a.customer_id = b.customer_id
where a.order_status not in ('unavailable','canceled')
and a.order_purchase_timestamp is not null
group by b.customer_unique_id
""").to_df()

In [167]:
df_customer_frequency

,customer_unique_id,no_order,account_age_month,order_per_month
0,57b2a98a409812fe9618067b6b8ebe4f,1,12,0.083333
1,21f748a16f4e1688a9014eb3ee6fa325,1,10,0.100000
2,a96d5cfa0d3181817e2b946f921ea021,1,13,0.076923
3,e9dd12dca17352644a959d9dea133935,2,8,0.285714
4,e079b18794454de9d2be5c12b4392294,2,17,0.117647
...,...,...,...,...
94985,d385d86ff15a721b213924f5f6b8316f,1,3,0.333333
94986,919decef1f13a9b93ed6cca4af6a8c2f,1,20,0.050000
94987,591373b45f940c1da643522fa4300753,1,12,0.083333
94988,6ff6f2913b97c58e8c0a4c96d8de6158,1,7,0.142857


7. monetary

In [168]:
df_customer_monetary = duckdb.query("""
select b.customer_unique_id
, count(distinct a.order_id) no_order
, sum(c.price) amount             
, max(datediff('month',cast(order_purchase_timestamp as datetime),cast('2018-11-01' as datetime))) account_age_month         
, sum(c.price)/count(distinct a.order_id) amount_per_order
, sum(c.price)/
max(datediff('month',cast(order_purchase_timestamp as datetime),cast('2018-11-01' as datetime))) amount_per_month
from df_order a
left join df_customer b
on a.customer_id = b.customer_id
left join df_order_item c
on a.order_id = c.order_id
where a.order_status not in ('unavailable','canceled')
and a.order_purchase_timestamp is not null
group by b.customer_unique_id
""").to_df()

In [169]:
df_customer_monetary

,customer_unique_id,no_order,amount,account_age_month,amount_per_order,amount_per_month
0,1175e95fb47ddff9de6b2b06188f7e0d,2,54.99,10,27.495,5.499000
1,2a46fb94aef5cbeeb850418118cee090,1,129.90,9,129.900,14.433333
2,2e6a42a9b5cbb0da62988694f18ee295,1,29.99,18,29.990,1.666111
3,ef07ba9aa5226f77264ffa5762b2280b,1,30.50,8,30.500,3.812500
4,e021e698833bdeb89dfef3acb2e91f37,1,223.90,22,223.900,10.177273
...,...,...,...,...,...,...
94985,81e481d14580da9f6a46a942be25a6ff,1,19.00,3,19.000,6.333333
94986,77d7e027eb0c7fe85780fc9fe35e0fbd,1,89.99,15,89.990,5.999333
94987,0ea123c30d764d3406572faf0fbddf21,1,159.90,7,159.900,22.842857
94988,93cc2577e09c0b0ba51e1831f45148dc,1,59.98,13,59.980,4.613846


# 3. Join data

In [170]:
df = duckdb.query("""
select a.customer_unique_id
, b.no_account
, c.age
, d.north, d.northeast, d.central_west, d.southeast, d.south
, e.day_after_last_pur
, f.order_per_month
, g.amount_per_order, g.amount_per_month
from df_customer_unique_id a
left join df_customer_no_acc b
on a.customer_unique_id = b.customer_unique_id
left join df_customer_age c
on a.customer_unique_id = c.customer_unique_id
left join df_customer_address d
on a.customer_unique_id = d.customer_unique_id
left join df_customer_recency e
on a.customer_unique_id = e.customer_unique_id
left join df_customer_frequency f
on a.customer_unique_id = f.customer_unique_id
left join df_customer_monetary g
on a.customer_unique_id = g.customer_unique_id
""").to_df()

In [171]:
df.head()

,customer_unique_id,no_account,age,north,northeast,central_west,southeast,south,day_after_last_pur,order_per_month,amount_per_order,amount_per_month
0,290c77bc529b7ac935b93aa66c333dc3,1,21.0,0,0,0,1,0,293.0,0.100000,289.00,28.900000
1,060e732b5b29e8181a18229c7b0b2b5e,1,32.0,0,0,0,1,0,166.0,0.166667,139.94,23.323333
2,5c2991dbd08bbf3cf410713c4de5a0b5,1,25.0,0,0,0,1,0,80.0,0.333333,119.90,39.966667
3,b6e99561fe6f34a55b0b7da92f8ed775,1,46.0,0,0,0,1,0,136.0,0.200000,7.09,1.418000
4,5f102dd37243f152aec3607970aad100,1,72.0,0,0,0,1,0,265.0,0.111111,49.00,5.444444


In [172]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96096 entries, 0 to 96095
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   customer_unique_id  96096 non-null  object 
 1   no_account          96096 non-null  int64  
 2   age                 95420 non-null  float64
 3   north               96096 non-null  int32  
 4   northeast           96096 non-null  int32  
 5   central_west        96096 non-null  int32  
 6   southeast           96096 non-null  int32  
 7   south               96096 non-null  int32  
 8   day_after_last_pur  94990 non-null  float64
 9   order_per_month     94990 non-null  float64
 10  amount_per_order    94983 non-null  float64
 11  amount_per_month    94983 non-null  float64
dtypes: float64(5), int32(5), int64(1), object(1)
memory usage: 7.0+ MB


In [173]:
df.isna().sum()

customer_unique_id       0
no_account               0
age                    676
north                    0
northeast                0
central_west             0
southeast                0
south                    0
day_after_last_pur    1106
order_per_month       1106
amount_per_order      1113
amount_per_month      1113
dtype: int64

In [174]:
df['day_after_last_pur'].fillna(0, inplace=True)
df['order_per_month'].fillna(0, inplace=True)
df['amount_per_order'].fillna(0, inplace=True)
df['amount_per_month'].fillna(0, inplace=True)

In [175]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96096 entries, 0 to 96095
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   customer_unique_id  96096 non-null  object 
 1   no_account          96096 non-null  int64  
 2   age                 95420 non-null  float64
 3   north               96096 non-null  int32  
 4   northeast           96096 non-null  int32  
 5   central_west        96096 non-null  int32  
 6   southeast           96096 non-null  int32  
 7   south               96096 non-null  int32  
 8   day_after_last_pur  96096 non-null  float64
 9   order_per_month     96096 non-null  float64
 10  amount_per_order    96096 non-null  float64
 11  amount_per_month    96096 non-null  float64
dtypes: float64(5), int32(5), int64(1), object(1)
memory usage: 7.0+ MB


In [176]:
df.dropna(inplace=True)

In [177]:
df

,customer_unique_id,no_account,age,north,northeast,central_west,southeast,south,day_after_last_pur,order_per_month,amount_per_order,amount_per_month
0,290c77bc529b7ac935b93aa66c333dc3,1,21.0,0,0,0,1,0,293.0,0.100000,289.00,28.900000
1,060e732b5b29e8181a18229c7b0b2b5e,1,32.0,0,0,0,1,0,166.0,0.166667,139.94,23.323333
2,5c2991dbd08bbf3cf410713c4de5a0b5,1,25.0,0,0,0,1,0,80.0,0.333333,119.90,39.966667
3,b6e99561fe6f34a55b0b7da92f8ed775,1,46.0,0,0,0,1,0,136.0,0.200000,7.09,1.418000
4,5f102dd37243f152aec3607970aad100,1,72.0,0,0,0,1,0,265.0,0.111111,49.00,5.444444
...,...,...,...,...,...,...,...,...,...,...,...,...
96091,bafe3b46fc75d1d1f93485d4a2d939c6,1,73.0,0,0,0,0,1,454.0,0.066667,59.90,3.993333
96092,71d5b68149301ba41f0a5445f990e742,1,64.0,0,0,0,1,0,189.0,0.142857,195.00,27.857143
96093,bc54474f8433c845bc7146b9b14e584e,1,57.0,0,0,1,0,0,281.0,0.100000,37.90,3.790000
96094,c9f9c202fad4743c180a0c0f6a5446c5,1,27.0,0,0,0,1,0,72.0,0.333333,39.90,13.300000


# 4. Exploratory data analysis (EDA)